In [ ]:
!pip install -q jmd_imagescraper
!pip install -Uqq fastbook

In [ ]:
import fastbook
from fastbook import *
from fastai.vision.widgets import *
from pathlib import Path
from jmd_imagescraper.core import *

In [ ]:
!rm -rf /content/plants

In [ ]:
root = "/content/plants"
duckduckgo_search(root, "dionaea", "dionaea", max_results=100)
duckduckgo_search(root, "drosera", "drosera", max_results=100)
duckduckgo_search(root, "nepenthes", "nepenthes", max_results=100)
duckduckgo_search(root, "random", "random images", max_results=100)

In [ ]:
path = Path('plants')
fns = get_image_files(path)

In [ ]:
failed = verify_images(fns)
failed.map(Path.unlink);

In [ ]:
plants = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=1),
    get_y=parent_label,
    item_tfms=Resize(128))
    
dls = plants.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=10, nrows=2)

In [ ]:
plants = plants.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = plants.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
plants = plants.new(item_tfms=Resize(128), batch_tfms=aug_transforms(max_lighting=0.6,max_rotate=180))
dls = plants.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
plants = plants.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms(max_lighting=0.6,max_rotate=180))
dls = plants.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(10, nrows=2)

In [ ]:
learn.export('model.pkl')